In [1]:
# TASKS
# Get rid of CNN and NYT and use sources more clearly on the left
## Don't want any David Brooks kinda stuff mixed in there
## NYT/CNN are 'centrist' kind of

In [140]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.svm import SVC
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import classification_report, confusion_matrix
import nltk
from nltk.stem.porter import PorterStemmer

In [3]:
bb = pd.read_csv('./jun20_bb_posts.csv', )
cnn = pd.read_csv('./jun20_cnn_posts.csv')
fox = pd.read_csv('./jun20_fox_posts.csv')
fox2 = pd.read_csv('./fox_posts_jun26.csv')
nr = pd.read_csv('./jun20_nr_posts.csv')
# nyt = pd.read_csv('./jun20_nyt_posts.csv')
vice = pd.read_csv('./jun20_vice_posts.csv')
dem = pd.read_csv('./jun21_demnow_posts.csv')
dem2 = pd.read_csv('./jun22_demnow_posts.csv')
dem3 = pd.read_csv('./jun23_demnow_posts.csv')
dem4 = pd.read_csv('./jun24_demnow_posts.csv')
dem5 = pd.read_csv('./jun25_demnow_posts.csv')
dem6 = pd.read_csv('./jun26_demnow_posts.csv')
inwa = pd.read_csv('./june20infowars.csv')
inwa2 = pd.read_csv('./june21infowars.csv')
inwa3 = pd.read_csv('./june22infowars.csv')
inwa4 = pd.read_csv('./june23infowars.csv')
inwa5 = pd.read_csv('./june24infowars.csv')
inwa6 = pd.read_csv('./june25infowars.csv')
inwa7 = pd.read_csv('./june26infowars.csv')
msnbc = pd.read_csv('./jun26_msnbc_posts.csv')
huff = pd.read_csv('./jun_24_huff_posts.csv')

In [4]:
bb['yes_right'] = 1
cnn['yes_right'] = 0
fox['yes_right'] = 1
fox2['yes_right'] = 1
nr['yes_right'] = 1
# nyt['yes_right'] = 0
vice['yes_right'] = 0
dem['yes_right'] = 0
dem2['yes_right'] = 0
dem3['yes_right'] = 0
dem4['yes_right'] = 0
dem5['yes_right'] = 0
dem6['yes_right'] = 0
inwa['yes_right'] = 1
inwa2['yes_right'] = 1
inwa3['yes_right'] = 1
inwa4['yes_right'] = 1
inwa5['yes_right'] = 1
inwa6['yes_right'] = 1
inwa7['yes_right'] = 1
msnbc['yes_right'] = 0
huff['yes_right'] = 0

In [5]:
text = pd.concat([bb, cnn, fox, fox2, nr, vice, dem, dem2, dem3, dem4, dem5, dem6, inwa, inwa2, inwa3, inwa4, inwa5, inwa6, inwa7, msnbc, huff])

In [6]:
text.drop(['Unnamed: 0', 'category', 'urlToImage'], axis=1, inplace=True)

In [7]:
text.isna().sum()

author         6342
description      68
publishedAt       0
source            0
title             1
url               0
yes_right         0
dtype: int64

In [8]:
no_desc = text.description.isna()

In [9]:
text.dropna(subset=['description','title'], inplace=True)

In [10]:
text.isna().sum()

author         6284
description       0
publishedAt       0
source            0
title             0
url               0
yes_right         0
dtype: int64

In [11]:
text.fillna('none', inplace=True)

In [12]:
text.shape

(40526, 7)

In [13]:
len(text.url.unique())

35786

In [14]:
text.drop_duplicates('url', inplace=True)

In [15]:
text.isna().sum().sum()

0

In [16]:
text.shape

(35786, 7)

In [17]:
text.reset_index(drop=True, inplace=True)

In [18]:
text.yes_right.value_counts()

0    19688
1    16098
Name: yes_right, dtype: int64

In [19]:
text.source.value_counts()

fox news           5833
msnbc              4950
huffington post    4950
vice news          4949
breitbart          4948
national review    4946
cnn                4776
infowars            371
democracy now        63
Name: source, dtype: int64

# Explorin

### Single Grams for Title

In [24]:
right = text['yes_right'] == 1
notright = text['yes_right'] == 0

In [25]:
right_title = text[right].title
right_desc = text[right].description
notright_title = text[notright].title
notright_desc = text[notright].description

In [26]:
stem = PorterStemmer()
cvec = CountVectorizer(preprocessor=stem.stem, ngram_range=(1,1), stop_words='english', min_df=10)

In [27]:
cvec.fit(right_title)
rt_counts = pd.DataFrame(cvec.transform(right_title).todense(),
                       columns=cvec.get_feature_names())
title_counts = rt_counts.sum(axis=0)
title_counts = pd.DataFrame(title_counts.sort_values(ascending = False), columns=['right'])

In [28]:
cvec = CountVectorizer(preprocessor=stem.stem, ngram_range=(1,1), stop_words='english', min_df=10)
cvec.fit(notright_title)
nrt_counts = pd.DataFrame(cvec.transform(notright_title).todense(),
                         columns=cvec.get_feature_names())
nrt_counts = nrt_counts.sum(axis=0)
title_counts['not right'] = nrt_counts

In [29]:
title_counts.head(20)

,right,not right
trump,2839,3872.0
new,1376,1079.0
fox,959,95.0
report,554,293.0
north,515,414.0
korea,498,401.0
says,405,492.0
kim,393,347.0
summit,378,283.0
day,358,250.0


In [30]:
title_counts.sort_values('not right', ascending=False).head(20)

,right,not right
trump,2839,3872.0
new,1376,1079.0
says,405,492.0
2018,206,427.0
house,327,425.0
north,515,414.0
korea,498,401.0
cohen,52,391.0
white,280,348.0
kim,393,347.0


### Single Grams for Description

In [31]:
cvec = CountVectorizer(preprocessor=stem.stem, ngram_range=(1,1), stop_words='english', min_df=10)
cvec.fit(right_desc)
rd_counts = pd.DataFrame(cvec.transform(right_desc).todense(),
                         columns=cvec.get_feature_names())
desc_counts = rd_counts.sum(axis=0)
desc_counts = pd.DataFrame(desc_counts.sort_values(ascending = False), columns=['right'])

In [32]:
cvec = CountVectorizer(preprocessor=stem.stem, ngram_range=(1,1), stop_words='english', min_df=10)
cvec.fit(notright_desc)
nrd_counts = pd.DataFrame(cvec.transform(notright_desc).todense(),
                         columns=cvec.get_feature_names())
nrd_counts = nrd_counts.sum(axis=0)
desc_counts['not right'] = nrd_counts

In [33]:
desc_counts.head(20)

,right,not right
trump,3670,4732.0
president,3096,2787.0
news,2877,857.0
fox,2545,179.0
new,1407,2049.0
said,1164,1089.0
house,1142,1029.0
donald,1102,1374.0
north,916,720.0
state,884,526.0


In [34]:
desc_counts.sort_values('not right', ascending=False).head(20)

,right,not right
trump,3670,4732.0
president,3096,2787.0
new,1407,2049.0
donald,1102,1374.0
said,1164,1089.0
house,1142,1029.0
cnn,188,958.0
news,2877,857.0
says,796,835.0
people,595,823.0


### BiGrams for Title

In [35]:
cvec = CountVectorizer(preprocessor=stem.stem, ngram_range=(2,2), stop_words='english', min_df=10)
cvec.fit(right_title)
rt_counts2 = pd.DataFrame(cvec.transform(right_title).todense(),
                         columns=cvec.get_feature_names())
title_counts2 = rt_counts2.sum(axis=0)
title_counts2 = pd.DataFrame(title_counts2.sort_values(ascending = False), columns=['right'])

In [36]:
cvec = CountVectorizer(preprocessor=stem.stem, ngram_range=(2,2), stop_words='english', min_df=10)
cvec.fit(notright_title)
nrt_counts2 = pd.DataFrame(cvec.transform(notright_desc).todense(),
                         columns=cvec.get_feature_names())
nrt_counts2 = nrt_counts2.sum(axis=0)
title_counts2['not right'] = nrt_counts2

In [37]:
title_counts2.head(15)

,right,not right
fox new,869,NaN
north korea,413,385.0
donald trump,301,1352.0
kim jong,180,414.0
president trump,175,898.0
white house,169,633.0
ig report,137,7.0
trump kim,120,61.0
judicial activism,96,NaN
day liberal,96,NaN


In [38]:
title_counts2.sort_values('not right', ascending=False).head(15)

,right,not right
donald trump,301,1352.0
president trump,175,898.0
white house,169,633.0
new york,94,417.0
kim jong,180,414.0
michael cohen,24,405.0
trump administration,63,391.0
north korea,413,385.0
north korean,54,254.0
special counsel,28,236.0


### BiGrams for Description

In [39]:
cvec = CountVectorizer(preprocessor=stem.stem, ngram_range=(2,2), stop_words='english', min_df=10)
cvec.fit(right_desc)
rd_counts2 = pd.DataFrame(cvec.transform(right_desc).todense(),
                         columns=cvec.get_feature_names())
desc_counts2 = rd_counts2.sum(axis=0)
desc_counts2 = pd.DataFrame(desc_counts2.sort_values(ascending = False), columns=['right'])

In [40]:
cvec = CountVectorizer(preprocessor=stem.stem, ngram_range=(2,2), stop_words='english', min_df=10)
cvec.fit(notright_desc)
nrd_counts2 = pd.DataFrame(cvec.transform(notright_desc).todense(),
                         columns=cvec.get_feature_names())
nrd_counts2 = nrd_counts2.sum(axis=0)
desc_counts2['not right'] = nrd_counts2

In [41]:
desc_counts2.head(15)

,right,not right
fox news,2243,101.0
president trump,1125,898.0
donald trump,1080,1352.0
president donald,927,673.0
white house,588,633.0
kim jong,471,414.0
north korea,460,385.0
new york,457,417.0
year old,312,198.0
north korean,311,254.0


In [42]:
desc_counts2.sort_values('not right',ascending=False).head(15)

,right,not right
donald trump,1080,1352.0
president trump,1125,898.0
president donald,927,673.0
white house,588,633.0
new york,457,417.0
kim jong,471,414.0
michael cohen,45,405.0
trump administration,243,391.0
north korea,460,385.0
north korean,311,254.0


### TriGrams for Title

In [47]:
cvec = CountVectorizer(preprocessor=stem.stem, ngram_range=(3,3), stop_words='english', min_df=5)
cvec.fit(right_title)
rt_counts3 = pd.DataFrame(cvec.transform(right_title).todense(),
                         columns=cvec.get_feature_names())
title_counts3 = rt_counts3.sum(axis=0)
title_counts3 = pd.DataFrame(title_counts3.sort_values(ascending = False), columns=['right'])

In [48]:
cvec = CountVectorizer(preprocessor=stem.stem, ngram_range=(3,3), stop_words='english', min_df=5)
cvec.fit(notright_title)
nrt_counts3 = pd.DataFrame(cvec.transform(notright_desc).todense(),
                         columns=cvec.get_feature_names())
nrt_counts3 = nrt_counts3.sum(axis=0)
title_counts3['not right'] = nrt_counts3

In [49]:
title_counts3.head(15)

,right,not right
liberal judicial activism,96,NaN
day liberal judicial,96,NaN
north korea summit,76,38.0
trump kim summit,65,8.0
say fox new,50,NaN
says fox new,34,NaN
police say fox,32,NaN
south china sea,25,13.0
border patrol agents,24,NaN
report fox new,24,NaN


In [50]:
title_counts3.sort_values('not right', ascending=False).head(15)

,right,not right
new york times,22,155.0
new york city,7,60.0
trump legal team,5,48.0
summit north korea,5,42.0
trump white house,8,41.0
trump kim jong,23,41.0
fe high school,7,39.0
santa fe high,9,38.0
north korea summit,76,38.0
donald trump jr,14,34.0


### TriGrams for Description

In [51]:
cvec = CountVectorizer(preprocessor=stem.stem, ngram_range=(3,3), stop_words='english', min_df=5)
cvec.fit(right_desc)
rd_counts3 = pd.DataFrame(cvec.transform(right_desc).todense(),
                         columns=cvec.get_feature_names())
desc_counts3 = rd_counts3.sum(axis=0)
desc_counts3 = pd.DataFrame(desc_counts3.sort_values(ascending = False), columns=['right'])

In [52]:
cvec = CountVectorizer(preprocessor=stem.stem, ngram_range=(3,3), stop_words='english', min_df=5)
cvec.fit(notright_desc)
nrd_counts3 = pd.DataFrame(cvec.transform(notright_desc).todense(),
                         columns=cvec.get_feature_names())
nrd_counts3 = nrd_counts3.sum(axis=0)
desc_counts3['not right'] = nrd_counts3

In [53]:
desc_counts3.head(15)

,right,not right
president donald trump,921,672.0
com wp content,227,NaN
com www nationalreview,227,NaN
www nationalreview com,227,NaN
nationalreview com wp,227,NaN
wp content uploads,227,NaN
wp com www,227,NaN
content uploads 2018,222,NaN
jpg fit 1024,208,NaN
fit 1024 2c597,206,NaN


In [54]:
desc_counts3.sort_values('not right',ascending=False).head(15)

,right,not right
president donald trump,921,672.0
north korean leader,101,158.0
new york times,115,155.0
leader kim jong,107,149.0
korean leader kim,90,133.0
special counsel robert,109,125.0
counsel robert mueller,107,122.0
fbi director james,60,81.0
director james comey,59,81.0
lawyer michael cohen,15,69.0


### QuadGrams for Title

In [60]:
cvec = CountVectorizer(preprocessor=stem.stem, ngram_range=(4,4), stop_words='english', min_df=3)
cvec.fit(right_title)
rt_counts4 = pd.DataFrame(cvec.transform(right_title).todense(),
                         columns=cvec.get_feature_names())
title_counts4 = rt_counts4.sum(axis=0)
title_counts4 = pd.DataFrame(title_counts4.sort_values(ascending = False), columns=['right'])

In [61]:
cvec = CountVectorizer(preprocessor=stem.stem, ngram_range=(4,4), stop_words='english', min_df=3)
cvec.fit(notright_title)
nrt_counts4 = pd.DataFrame(cvec.transform(notright_desc).todense(),
                         columns=cvec.get_feature_names())
nrt_counts4 = nrt_counts4.sum(axis=0)
title_counts4['not right'] = nrt_counts4

In [62]:
title_counts4.head(15)

,right,not right
day liberal judicial activism,96,NaN
police say fox new,32,NaN
liberal judicial activism march,20,NaN
liberal judicial activism february,19,NaN
liberal judicial activism april,19,NaN
liberal judicial activism june,14,NaN
report says fox new,12,NaN
fashion notes melania trump,9,NaN
national review summer internship,9,NaN
north korea kim jong,9,NaN


In [63]:
title_counts4.sort_values('not right', ascending=False).head(15)

,right,not right
santa fe high school,7,38.0
new york attorney general,6,18.0
white house correspondents dinner,5,12.0
attorney general eric schneiderman,3,7.0
summit north korea kim,3,7.0
trump meeting kim jong,6,3.0
dhs secretary kirstjen nielsen,3,3.0
designer kate spade dead,4,1.0
elon musk boring company,4,1.0
waffle house shooting suspect,3,1.0


### QuadGrams for Description

In [95]:
cvec = CountVectorizer(preprocessor=stem.stem, ngram_range=(4,4), stop_words=cust_stop_words, min_df=5)
cvec.fit(right_desc)
rd_counts4 = pd.DataFrame(cvec.transform(right_desc).todense(),
                         columns=cvec.get_feature_names())
desc_counts4 = rd_counts4.sum(axis=0)
desc_counts4 = pd.DataFrame(desc_counts4.sort_values(ascending = False), columns=['right'])

In [96]:
cvec = CountVectorizer(preprocessor=stem.stem, ngram_range=(4,4), stop_words=cust_stop_words, min_df=5)
cvec.fit(notright_desc)
nrd_counts4 = pd.DataFrame(cvec.transform(notright_desc).todense(),
                         columns=cvec.get_feature_names())
nrd_counts4 = nrd_counts4.sum(axis=0)
desc_counts4['not right'] = nrd_counts4

In [97]:
desc_counts4.head(15)

,right,not right
com wp content uploads,227,NaN
com com wp content,227,NaN
wp com com wp,227,NaN
wp content uploads 2018,222,NaN
fit 1024 2c597 ssl,206,NaN
jpg fit 1024 2c597,200,NaN
dictator kim jong un,125,21.0
special counsel robert mueller,107,122.0
leader kim jong un,107,148.0
north korean dictator kim,94,17.0


In [67]:
desc_counts4.sort_values('not right', ascending=False).head(15)

,right,not right
north korean leader kim,90,133.0
korean leader kim jong,89,132.0
special counsel robert mueller,107,122.0
fbi director james comey,59,81.0
attorney general jeff sessions,46,63.0
white house press secretary,65,47.0
secretary state mike pompeo,53,47.0
trump north korean leader,26,45.0
santa fe high school,20,38.0
house press secretary sarah,40,35.0


### Entire Set

#### Modeling

In [130]:
X = text[['description', 'title']]
# X = text['description']
y = text['yes_right']

In [131]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

# Simple CountVec

In [94]:
cust_stop_words = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't", 'cnn', 'national review', 'huffpost','fox','reuters','ap','vice','breitbart', 'nationalreview', 'www', 'content uploads']

In [120]:
cvec = CountVectorizer(strip_accents='ascii', ngram_range=(2,3), stop_words=cust_stop_words, max_features=1250, min_df=5)

In [121]:
cvec.fit(X_train, y_train)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=1250, min_df=5,
        ngram_range=(2, 3), preprocessor=None,
        stop_words=['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs',...'huffpost', 'fox', 'reuters', 'ap', 'vice', 'breitbart', 'nationalreview', 'www', 'content uploads'],
        strip_accents='ascii', token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [122]:
X_feats = pd.DataFrame(cvec.transform(X_train).todense(),
                       columns=cvec.get_feature_names())

In [123]:
word_counts = X_feats.sum(axis=0)
word_counts.sort_values(ascending = False).head(20)

donald trump              1696
president trump           1332
president donald          1206
president donald trump    1113
white house                956
kim jong                   668
new york                   666
jong un                    644
kim jong un                644
north korea                615
trump administration       433
north korean               417
united states              396
year old                   385
michael cohen              311
special counsel            292
rachel maddow              255
attorney general           242
robert mueller             230
new york times             204
dtype: int64

In [124]:
cv_train = cvec.transform(X_train).todense()
cv_test = cvec.transform(X_test).todense()

In [125]:
lr = LogisticRegression(random_state=42)
lr.fit(cv_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=42, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [126]:
lr.score(cv_train, y_train)

0.70159096836692869

In [127]:
lr.score(cv_test, y_test)

0.67687493014418243

In [128]:
cfs = lr.coef_[0]
fts = cvec.get_feature_names()

In [129]:
pd.DataFrame(
    list(zip(fts, np.abs(cfs), cfs)),
    columns=['feat','abs','coef']).sort_values('abs',ascending=False).head(25)

,feat,abs,coef
55,amp friends,3.397190,3.397190
48,ali velshi,3.079380,-3.079380
1094,times local,3.033343,3.033343
360,foxnews com,2.874136,2.874136
1047,stephanie ruhle,2.824023,-2.824023
693,national review,2.709089,2.709089
298,far left,2.671317,2.671317
719,news contributor,2.564535,2.564535
782,panel discusses,2.563279,-2.563279
717,news channels,2.531941,2.531941


### PorterStemmer and TfIdf

In [143]:
# def stemming_tokenizer(text):
#     stemmer = PorterStemmer()
#     return [stemmer.stem(w) for w in word_tokenize(text)]

In [146]:
stemmer=PorterStemmer()

In [147]:
tf = TfidfVectorizer(
    strip_accents='ascii',
    tokenizer=stemmer.stem,
    ngram_range=(1,3),
    stop_words=cust_stop_words
)

In [148]:
tf.fit(X_train, y_train)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 3), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs',...t", 'cnn', 'national review', 'huffpost', 'fox', 'york times', 'reuters', 'ap', 'vice', 'breitbart'],
        strip_accents='ascii', sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=<bound method PorterStemmer.stem of <PorterStemmer>>,
        use_idf=True, vocabulary=None)

In [149]:
tf_train = tf.transform(X_train)
tf_test = tf.transform(X_test)

In [151]:
lr = LogisticRegression(random_state=42)

In [153]:
parameters = {
    'penalty': ['l2','l1'],
    'C': [1.0,0.8,0.6]
}

grid = GridSearchCV(lr, parameters, scoring='accuracy')

In [154]:
grid.fit(tf_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=42, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'penalty': ['l2', 'l1'], 'C': [1.0, 0.8, 0.6]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=0)

In [155]:
grid.score(tf_test, y_test)

0.73625933831376733

# Pipe

In [132]:
class DfExtract(BaseEstimator, TransformerMixin):
    def __init__(self, column):
        self.column = column
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        if len(self.column) > 1:
            return pd.DataFrame(X[self.column])
        else:
            return pd.Series(X[self.column[0]]) 

In [133]:
pipe = Pipeline([
    ('features', FeatureUnion([
        ('tit', Pipeline([
            ('ext', DfExtract(['title'])),
#             ('cvec', CountVectorizer(ngram_range=(1,2), stop_words='english')),
            ('tf', TfidfVectorizer(stop_words=cust_stop_words, ngram_range=(1,3))) #was transformer
        ])),
        ('desc', Pipeline([
            ('ext', DfExtract(['description'])),
#             ('cvec', CountVectorizer(ngram_range=(1,2), stop_words='english')),
            ('tf', TfidfVectorizer(stop_words=cust_stop_words, ngram_range=(1,3))) # was transformer
        ]))
    ])),
    ('lr', LogisticRegression(random_state=42))
#     ('svc', SVC(random_state=42))
#     ('bnb', BernoulliNB())
])

In [144]:
params = {
#     'cvec__max_df': (0.5, 0.75, 1.0),
#     'tf__norm': ['l1', 'l2', None],
#     'tf__use_idf': [True, False],
#     'tf__max_df': [1.0, 5],
    'lr__penalty': ['l1','l2'],
    'lr__C': [0.75, 1.0],
#     'svc__kernel': ['rbf', 'sigmoid', 'poly'],
#     'bnb__alpha': [1.0, 0.75, 0.5, 0.35] 
}

gs = GridSearchCV(pipe, params, scoring='roc_auc')

In [145]:
gs.fit(X_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('tit', Pipeline(memory=None,
     steps=[('ext', DfExtract(column=['title'])), ('tf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
   ...alty='l2', random_state=42, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'lr__penalty': ['l1', 'l2'], 'lr__C': [0.75, 1.0]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=0)

In [146]:
gs.score(X_test, y_test)

0.8706999376610528

In [147]:
gs.best_params_

{'lr__C': 1.0, 'lr__penalty': 'l2'}

In [148]:
coefs = list(gs.best_estimator_.steps[1][1].coef_[0])

In [149]:
len(coefs)

869440

In [150]:
print(classification_report(y_test, gs.predict(X_test)))

             precision    recall  f1-score   support

          0       0.80      0.82      0.81      4922
          1       0.77      0.74      0.76      4025

avg / total       0.78      0.78      0.78      8947



In [151]:
confusion_matrix(y_test, gs.predict(X_test)) # false positives

array([[4036,  886],
       [1039, 2986]])

In [152]:
# gs.best_estimator_.steps[0][1].get_feature_names()

In [153]:
# lr_coefs = pd.DataFrame(
#     list(
#         zip(X_test.columns, np.abs(
#             coefs))), columns=['feature','coef_abs'])
# lr_coefs.sort_values('coef_abs', ascending=False, inplace=True)

----------------------------------------------------------------------------

In [154]:
def manual_test():
    title = input('Article title: \n')
    desc = input('Article description: \n')
    df = [{'title': '{}'.format(title),
                'description': '{}'.format(desc)
                }]
    df=pd.DataFrame(df)
    pred = gs.predict(df)
    if pred == 1:
        print("Result: Right wing")
    else:
        print("Result: Not right wing")

In [155]:
manual_test()

Article title: 
Seventeen states sue Trump administration over family separations
Article description: 
States including New York, California and Washington filed lawsuits accusing government of denying people their rights
Result: Not right wing


In [156]:
manual_test()

Article title: 
Democratic candidates revolt against Pelosi in House races across the country
Article description: 
Anyone looking for signs that Nancy Pelosi has lost clout within the Democratic ranks this cycle need only catch a glimpse of last week’s candidate forum in New Hampshire’s 1st Congressional District
Result: Not right wing
